In [189]:
!pip install geopandas

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 245, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1368, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/protocol.py", line 36

In [190]:
!hdfs dfs -mkdir /raw

mkdir: `/raw': File exists


In [191]:
!hdfs dfs -mkdir /raw/new

mkdir: `/raw/new': File exists


In [192]:
!hdfs dfs -mkdir /data

mkdir: `/data': File exists


In [193]:

!hdfs dfs -mkdir /raw/processed

mkdir: `/raw/processed': File exists


In [32]:
#!hadoop fs -rm -r /raw/new/*.parquet

In [33]:

#!hadoop fs -rm -r /raw/processed/*.parquet

In [194]:
!hdfs dfs -ls /raw/new/

Found 4 items
drwxr-xr-x   - root supergroup          0 2024-06-21 01:58 /raw/new/data_raw_21_06_2024__01_58_16.parquet
drwxr-xr-x   - root supergroup          0 2024-06-21 02:02 /raw/new/data_raw_21_06_2024__02_02_03.parquet
drwxr-xr-x   - root supergroup          0 2024-06-21 02:05 /raw/new/data_raw_21_06_2024__02_05_51.parquet
drwxr-xr-x   - root supergroup          0 2024-06-21 02:09 /raw/new/data_raw_21_06_2024__02_09_38.parquet


In [35]:
#!hadoop fs -rm -r /raw/data_raw.parquet

In [195]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time
import json
from datetime import datetime

spark = SparkSession \
  .builder \
  .appName("streaming") \
  .master("local[*]") \
  .getOrCreate()

spark

In [196]:
schema = ArrayType(StructType([
    StructField("object_id", IntegerType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("date", StringType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("employee_id", IntegerType(), True),
    StructField("quantity_products", IntegerType(), True),
    StructField("order_id", StringType(), True)
]))

static_schema = StructType([
    StructField("object_id", IntegerType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("date", StringType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("employee_id", IntegerType(), True),
    StructField("quantity_products", IntegerType(), True),
    StructField("order_id", StringType(), True)
])

In [197]:
static_df = spark.createDataFrame([], static_schema)
while True:
    streaming_df = spark.readStream.format("socket").option("host", "localhost").option("port", "8000").load()
    json_df = streaming_df.select(from_json(col("value"), schema).alias("data"))
    json_df = json_df.selectExpr("explode(data) as dict").select("dict.*")
    writing_df = json_df.writeStream.format("memory").queryName("socketData").outputMode("update").start()
    static_stream_df = spark.sql("SELECT * FROM socketData")
    static_df = static_df.union(static_stream_df)
    writing_df.awaitTermination(2)
    writing_df.stop()
    if static_df.count() > 50:
        now = datetime.now()
        now = now.strftime("%d_%m_%Y__%H_%M_%S")
        ruta = '/raw/new/data_raw_'+now+'.parquet'
        print(ruta)
        static_df.write.mode('append').parquet(ruta)
        static_df = spark.createDataFrame([], static_schema)
    time.sleep(30)

/raw/new/data_raw_21_06_2024__05_22_48.parquet


24/06/21 05:26:02 ERROR streaming.MicroBatchExecution: Query socketData [id = 968dc90d-01b5-40cf-b953-99d6edfef47e, runId = 57ea80b1-24ad-4eac-a0e2-75beb23d03d9] terminated with error
java.net.ConnectException: Connection refused (Connection refused)
	at java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:188)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:392)
	at java.net.Socket.connect(Socket.java:607)
	at java.net.Socket.connect(Socket.java:556)
	at java.net.Socket.<init>(Socket.java:452)
	at java.net.Socket.<init>(Socket.java:229)
	at org.apache.spark.sql.execution.streaming.sources.TextSocketMicroBatchStream.initialize(TextSocketMicroBatchStream.scala:71)
	at org.apache.spark.sql.execution.streaming.sources.TextSocketMicroBatchS

StreamingQueryException: Connection refused (Connection refused)
=== Streaming Query ===
Identifier: socketData [id = 968dc90d-01b5-40cf-b953-99d6edfef47e, runId = 57ea80b1-24ad-4eac-a0e2-75beb23d03d9]
Current Committed Offsets: {}
Current Available Offsets: {TextSocketV2[host: localhost, port: 8000]: -1}

Current State: ACTIVE
Thread State: RUNNABLE

Logical Plan:
WriteToMicroBatchDataSource org.apache.spark.sql.execution.streaming.sources.MemoryStreamingWrite@a305d6f
+- Project [dict#71128.object_id AS object_id#71130, dict#71128.latitude AS latitude#71131, dict#71128.longitude AS longitude#71132, dict#71128.date AS date#71133, dict#71128.customer_id AS customer_id#71134, dict#71128.employee_id AS employee_id#71135, dict#71128.quantity_products AS quantity_products#71136, dict#71128.order_id AS order_id#71137]
   +- Project [dict#71128]
      +- Generate explode(data#71125), false, [dict#71128]
         +- Project [from_json(ArrayType(StructType(StructField(object_id,IntegerType,true

In [198]:
df = spark.read.parquet("/raw/new/*.parquet")
print("Tabla estatica:", df.count())
df = df.orderBy(desc("order_id"))
df.show(truncate=False)

Tabla estatica: 269
+---------+------------------+------------------+-------------------+-----------+-----------+-----------------+------------------------------------+
|object_id|latitude          |longitude         |date               |customer_id|employee_id|quantity_products|order_id                            |
+---------+------------------+------------------+-------------------+-----------+-----------+-----------------+------------------------------------+
|336      |6.312877773625606 |-75.67142513127672|21/06/2024 02:09:36|8837       |9726       |577              |d8b9b417-b098-4344-b137-362894e4dckh|
|339      |6.248162954925148 |-75.49213858646789|21/06/2024 02:09:36|2381       |6696       |476              |d8b9b417-b098-4344-b137-362894e4dckg|
|328      |6.260502879589433 |-75.55631439510852|21/06/2024 02:09:36|7311       |9726       |252              |d8b9b417-b098-4344-b137-362894e4dckf|
|322      |6.293711329824756 |-75.55888626472442|21/06/2024 02:09:36|8190       |3455 

In [199]:
paths = [
    'file:///workspace/base.data/50001.parquet',
    'file:///workspace/base.data/customers.parquet',
    'file:///workspace/base.data/employees.parquet',
    'file:///workspace/base.data/medellin_neighborhoods.parquet'
]

hdfs_destination = "hdfs:////data/"


Función para mover archivos a HDFS

In [200]:
def move_to_hdfs(file_paths, hdfs_dest):
    for path in file_paths:
        # Leer el archivo local
        df = spark.read.parquet(path)
        
        # Extraer el nombre del archivo para usarlo como nombre de archivo en HDFS
        file_name = path.split("/")[-1]
        
        # Escribir en HDFS
        df.write.parquet(hdfs_dest + file_name, mode="overwrite")
        
        print(f"Archivo {file_name} movido a {hdfs_dest}")

# Llamar a la función para mover archivos
move_to_hdfs(paths, hdfs_destination)

Archivo 50001.parquet movido a hdfs:////data/
Archivo customers.parquet movido a hdfs:////data/
Archivo employees.parquet movido a hdfs:////data/
Archivo medellin_neighborhoods.parquet movido a hdfs:////data/


Listar Archivos en HDFS desde Python

In [201]:
from subprocess import Popen, PIPE

# Comando para listar archivos en HDFS
command = ['hadoop', 'fs', '-ls', '/raw' ]

# Ejecutar el comando y capturar la salida
process = Popen(command, stdout=PIPE, stderr=PIPE)
stdout, stderr = process.communicate()

# Decodificar la salida y mostrarla
output = stdout.decode()
print(output)


Found 6 items
drwxr-xr-x   - root supergroup          0 2024-06-19 20:56 /raw/50001.parquet
drwxr-xr-x   - root supergroup          0 2024-06-19 20:56 /raw/customers.parquet
drwxr-xr-x   - root supergroup          0 2024-06-19 20:56 /raw/employees.parquet
drwxr-xr-x   - root supergroup          0 2024-06-19 20:56 /raw/medellin_neighborhoods.parquet
drwxr-xr-x   - root supergroup          0 2024-06-21 05:22 /raw/new
drwxr-xr-x   - root supergroup          0 2024-06-21 01:52 /raw/processed



In [202]:
!hdfs dfs -ls /raw/new

Found 5 items
drwxr-xr-x   - root supergroup          0 2024-06-21 01:58 /raw/new/data_raw_21_06_2024__01_58_16.parquet
drwxr-xr-x   - root supergroup          0 2024-06-21 02:02 /raw/new/data_raw_21_06_2024__02_02_03.parquet
drwxr-xr-x   - root supergroup          0 2024-06-21 02:05 /raw/new/data_raw_21_06_2024__02_05_51.parquet
drwxr-xr-x   - root supergroup          0 2024-06-21 02:09 /raw/new/data_raw_21_06_2024__02_09_38.parquet
drwxr-xr-x   - root supergroup          0 2024-06-21 05:22 /raw/new/data_raw_21_06_2024__05_22_48.parquet


In [203]:
!hdfs dfs -ls /data


Found 4 items
drwxr-xr-x   - root supergroup          0 2024-06-21 05:26 /data/50001.parquet
drwxr-xr-x   - root supergroup          0 2024-06-21 05:26 /data/customers.parquet
drwxr-xr-x   - root supergroup          0 2024-06-21 05:26 /data/employees.parquet
drwxr-xr-x   - root supergroup          0 2024-06-21 05:26 /data/medellin_neighborhoods.parquet


In [204]:
#!hdfs dfs -rm -r /raw/new/50001.parquet
#!hdfs dfs -rm -r /raw/new/customers.parquet
#!hdfs dfs -rm -r /raw/new/employees.parquet
#!hdfs dfs -rm -r /raw/new/medellin_neighborhoods.parquet


In [205]:
# Definir la ruta base en HDFS
hdfs_base_path = 'hdfs://localhost:9000'

# Leer cada archivo Parquet
df_50001 = spark.read.parquet(hdfs_base_path + '/data/50001.parquet')
df_customers = spark.read.parquet(hdfs_base_path + '/data/customers.parquet')
df_data_raw = spark.read.parquet(hdfs_base_path + '/raw/new/*.parquet')
df_employees = spark.read.parquet(hdfs_base_path + '/data/employees.parquet')
df_neighborhoods = spark.read.parquet(hdfs_base_path + '/data/medellin_neighborhoods.parquet')


In [206]:
df_data_raw.show(10)


+---------+------------------+------------------+-------------------+-----------+-----------+-----------------+--------------------+
|object_id|          latitude|         longitude|               date|customer_id|employee_id|quantity_products|            order_id|
+---------+------------------+------------------+-------------------+-----------+-----------+-----------------+--------------------+
|      337| 6.272252523303907|-75.64754590534695|21/06/2024 01:55:01|       6253|       1679|              225|d8b9b417-b098-434...|
|      330| 6.228334667224176|-75.56989476031579|21/06/2024 01:55:01|       5687|       3830|              294|d8b9b417-b098-434...|
|      341|6.2644164362595784|-75.69769593180834|21/06/2024 01:55:01|       8837|       2232|              422|d8b9b417-b098-434...|
|      337| 6.312778670415543|-75.66684760175428|21/06/2024 01:55:01|       2390|       9438|              727|d8b9b417-b098-434...|
|      335| 6.237990075558335|-75.58065203777282|21/06/2024 01:55:01|

In [207]:
df_50001.show()

+--------+----------+----------+----------+----------+--------------------+
|DPTOMPIO|DPTO_CCDGO|MPIO_CCDGO|MPIO_CNMBR|MPIO_CCNCT|            geometry|
+--------+----------+----------+----------+----------+--------------------+
|   05001|        05|       001|  MEDELLÍN|     05001|[01 03 00 00 00 0...|
+--------+----------+----------+----------+----------+--------------------+



In [208]:
df_neighborhoods.show(5)

+--------+------+----------+--------------+-----------------+---------------------------+--------------+----------------+------------------+--------------------+
|OBJECTID|CODIGO|    NOMBRE|IDENTIFICACION|LIMITEMUNICIPIOID|SUBTIPO_COMUNACORREGIMIENTO|LINK_DOCUMENTO|       SHAPEAREA|          SHAPELEN|            geometry|
+--------+------+----------+--------------+-----------------+---------------------------+--------------+----------------+------------------+--------------------+
|     321|    01|   POPULAR|      COMUNA 1|              001|                          1|          null|3098289.60257159| 9604.987826371042|[01 03 00 00 00 0...|
|     322|    02|SANTA CRUZ|      COMUNA 2|              001|                          1|          null|2195874.52580248| 8597.714448746181|[01 03 00 00 00 0...|
|     323|    03|  MANRIQUE|      COMUNA 3|              001|                          1|          null|5096746.29132065|  12078.2371083362|[01 03 00 00 00 0...|
|     324|    04|  ARANJUEZ|

In [209]:
df_employees.show(5)

+-----------+----------------+--------------+--------------------+--------------------+---------+
|employee_id|            name|         phone|               email|             address|comission|
+-----------+----------------+--------------+--------------------+--------------------+---------+
|       3830|Shaeleigh Turner|1-382-217-5724|pellentesque.ultr...| Ap #497-3659 Eu St.|     0.06|
|       8362|  Catherine King|1-721-878-1085|  sed@localstack.com|Ap #897-2636 Enim...|     0.07|
|       6696|    Patricia Cox|1-265-643-2312|imperdiet.erat.no...|Ap #775-1599 Sed Av.|     0.04|
|       1482|   Elijah Parker|1-960-392-6387|blandit.congue@lo...|P.O. Box 351, 382...|     0.13|
|       9435|    Ryan Nichols|1-746-416-6687|porttitor.tellus....|P.O. Box 829, 407...|     0.18|
+-----------+----------------+--------------+--------------------+--------------------+---------+
only showing top 5 rows



In [210]:
# Mostrar el esquema del DataFrame
df_data_raw.printSchema()

# Contar la cantidad de eventos
event_count = df_data_raw.count()
# Mostrar la cantidad de eventos
print(f"La cantidad de eventos es: {event_count}")

root
 |-- object_id: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- date: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- employee_id: integer (nullable = true)
 |-- quantity_products: integer (nullable = true)
 |-- order_id: string (nullable = true)

La cantidad de eventos es: 269


In [211]:
#Dejar las tablas en bronze con HIVE
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark SQL Hive Integration") \
    .config("spark.sql.warehouse.dir", "/user/hive/warehouse") \
    .enableHiveSupport() \
    .getOrCreate()

In [212]:
# Aumentar el límite de la tasa de datos de IOPub
from notebook.services.config import ConfigManager
cm = ConfigManager().update('notebook', {
    'ServerApp': {
        'iopub_data_rate_limit': 10000000
    }
})


In [213]:
# Crear la base de datos bronze si no existe
spark.sql("CREATE DATABASE IF NOT EXISTS bronze")

DataFrame[]

In [214]:
# Guardar los datos como una tabla en la base de datos bronze
df_data_raw.write.mode('append').saveAsTable('bronze.almacenamiento_bronze_nuevo')


In [215]:
# Consulta para seleccionar todos los datos de la tabla
resultados = spark.sql("SELECT * FROM bronze.almacenamiento_bronze_nuevo")

# Mostrar los primeros registros de los resultados
resultados.show()


+---------+-----------------+------------------+-------------------+-----------+-----------+-----------------+--------------------+
|object_id|         latitude|         longitude|               date|customer_id|employee_id|quantity_products|            order_id|
+---------+-----------------+------------------+-------------------+-----------+-----------+-----------------+--------------------+
|      334|6.222446938596848|-75.56855864645483|21/06/2024 01:59:20|       8657|       2232|              392|d8b9b417-b098-434...|
|      339|6.216340691862865|-75.52325545011696|21/06/2024 01:59:52|       3118|       4942|              539|d8b9b417-b098-434...|
|      338|6.234301858827036| -75.6168586442365|21/06/2024 01:59:52|       5624|       9438|              540|d8b9b417-b098-434...|
|      337|6.260951370843877|-75.66692073981508|21/06/2024 01:59:52|       8184|       2470|              444|d8b9b417-b098-434...|
|      339|6.251539356032007|-75.53244101782771|21/06/2024 01:59:52|       2

In [216]:
# Verificar que la tabla está guardada en almacenamiento_bronze
spark.sql("SELECT * FROM bronze.almacenamiento_bronze_nuevo").show(truncate=False)

+---------+-----------------+------------------+-------------------+-----------+-----------+-----------------+------------------------------------+
|object_id|latitude         |longitude         |date               |customer_id|employee_id|quantity_products|order_id                            |
+---------+-----------------+------------------+-------------------+-----------+-----------+-----------------+------------------------------------+
|334      |6.222446938596848|-75.56855864645483|21/06/2024 01:59:20|8657       |2232       |392              |d8b9b417-b098-4344-b137-362894e4dceu|
|339      |6.216340691862865|-75.52325545011696|21/06/2024 01:59:52|3118       |4942       |539              |d8b9b417-b098-4344-b137-362894e4dcev|
|338      |6.234301858827036|-75.6168586442365 |21/06/2024 01:59:52|5624       |9438       |540              |d8b9b417-b098-4344-b137-362894e4dcew|
|337      |6.260951370843877|-75.66692073981508|21/06/2024 01:59:52|8184       |2470       |444              |d8

In [217]:
# Crear la base de datos bronze si no existe
spark.sql("CREATE DATABASE IF NOT EXISTS bronze")
# Guardar los datos como una tabla en la base de datos municipio
df_50001.write.mode('append').saveAsTable('bronze.municipio_nuevo')
# Verificar que la tabla está guardada en municipio
#spark.sql("SELECT * FROM bronze.municipio_nuevo").show(truncate=False, n=20)


In [218]:
# Crear la base de datos bronze si no existe
spark.sql("CREATE DATABASE IF NOT EXISTS bronze")
# Guardar los datos como una tabla en la base de datos neighborhoods
df_neighborhoods.write.mode('overwrite').saveAsTable('bronze.neighborhoods_nuevo')
# Verificar que la tabla está guardada en barrios y mostrar solo las primeras 20 filas de algunas columnas
spark.sql("SELECT CODIGO, NOMBRE, IDENTIFICACION, LIMITEMUNICIPIOID,SUBTIPO_COMUNACORREGIMIENTO  FROM bronze.neighborhoods_nuevo").show(truncate=False, n=5)


+------+----------+--------------+-----------------+---------------------------+
|CODIGO|NOMBRE    |IDENTIFICACION|LIMITEMUNICIPIOID|SUBTIPO_COMUNACORREGIMIENTO|
+------+----------+--------------+-----------------+---------------------------+
|01    |POPULAR   |COMUNA 1      |001              |1                          |
|02    |SANTA CRUZ|COMUNA 2      |001              |1                          |
|03    |MANRIQUE  |COMUNA 3      |001              |1                          |
|04    |ARANJUEZ  |COMUNA 4      |001              |1                          |
|05    |CASTILLA  |COMUNA 5      |001              |1                          |
+------+----------+--------------+-----------------+---------------------------+
only showing top 5 rows



In [219]:
# Crear la base de datos bronze si no existe
spark.sql("CREATE DATABASE IF NOT EXISTS bronze")
# Guardar los datos como una tabla en la base de datos clientes
df_customers.write.mode('overwrite').saveAsTable('bronze.customers_nuevo')
# Verificar que la tabla está guardada en clientes
spark.sql("SELECT * FROM bronze.customers_nuevo").show(truncate=False)

+-----------+--------------------+--------------+------------------------------+-------------------------------+
|customer_id|name                |phone         |email                         |address                        |
+-----------+--------------------+--------------+------------------------------+-------------------------------+
|4758       |Callie Reyes        |1-765-410-5785|magnis.dis@protonmail.ca      |151-4553 Interdum Road         |
|5379       |Elizabeth Washington|1-955-634-5542|vel@google.edu                |4063 Nunc St.                  |
|8111       |Hasad Wright        |1-324-830-5595|sed.auctor@aol.org            |Ap #625-8512 Non Rd.           |
|9258       |Kirk Watts          |1-578-784-1146|laoreet.ipsum@protonmail.org  |211-2213 Pede St.              |
|9142       |Cally Robbins       |1-887-472-0478|at.augue.id@google.com        |Ap #287-6324 A, Av.            |
|5041       |Benedict Underwood  |1-138-146-9856|ante.ipsum@yahoo.ca           |2927 Velit Rd.  

In [220]:
# Crear la base de datos bronze si no existe
spark.sql("CREATE DATABASE IF NOT EXISTS bronze")
# Guardar los datos como una tabla en la base de datos empleados
df_employees.write.mode('overwrite').saveAsTable('bronze.employees_nuevo')
# Verificar que la tabla está guardada en empleados
spark.sql("SELECT * FROM bronze.employees_nuevo").show(truncate=False)

+-----------+-----------------+--------------+--------------------------------------+-----------------------------+---------+
|employee_id|name             |phone         |email                                 |address                      |comission|
+-----------+-----------------+--------------+--------------------------------------+-----------------------------+---------+
|3830       |Shaeleigh Turner |1-382-217-5724|pellentesque.ultricies@localstack.com |Ap #497-3659 Eu St.          |0.06     |
|8362       |Catherine King   |1-721-878-1085|sed@localstack.com                    |Ap #897-2636 Enim Av.        |0.07     |
|6696       |Patricia Cox     |1-265-643-2312|imperdiet.erat.nonummy@localstack.com |Ap #775-1599 Sed Av.         |0.04     |
|1482       |Elijah Parker    |1-960-392-6387|blandit.congue@localstack.com         |P.O. Box 351, 3827 Dolor. Ave|0.13     |
|9435       |Ryan Nichols     |1-746-416-6687|porttitor.tellus.non@localstack.com   |P.O. Box 829, 4074 Et Rd.    |0.1

# En Bronze se dejaron las tablas en Hive: nombre BD= bronze con tablas 1.almacenamiento_bronze 2.municipio 3.neighborhoods 4.customers y 5.employees

##### CAPA SILVER ###########

In [221]:
# Listar las tablas en la base de datos eventos_bronze
tables_bronze = spark.sql("SHOW TABLES IN bronze")
tables_bronze.show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|  bronze|almacenamiento_br...|      false|
|  bronze|     customers_nuevo|      false|
|  bronze|     employees_nuevo|      false|
|  bronze|     municipio_nuevo|      false|
|  bronze| neighborhoods_nuevo|      false|
|        |          socketdata|       true|
+--------+--------------------+-----------+



In [222]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, to_timestamp, month, dayofmonth, hour, minute, second, date_format


In [223]:
spark = SparkSession.builder \
    .appName("Transformación Bronze a Silver") \
    .getOrCreate()


In [224]:
bronze_path_hdfs = 'hdfs://localhost:9000'


In [225]:
df_bronze = spark.read.parquet(bronze_path_hdfs + '/raw/new/*.parquet')


In [226]:
df_bronze.printSchema()
df_bronze.show(10, truncate=False)


root
 |-- object_id: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- date: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- employee_id: integer (nullable = true)
 |-- quantity_products: integer (nullable = true)
 |-- order_id: string (nullable = true)

+---------+------------------+------------------+-------------------+-----------+-----------+-----------------+------------------------------------+
|object_id|latitude          |longitude         |date               |customer_id|employee_id|quantity_products|order_id                            |
+---------+------------------+------------------+-------------------+-----------+-----------+-----------------+------------------------------------+
|337      |6.272252523303907 |-75.64754590534695|21/06/2024 01:55:01|6253       |1679       |225              |d8b9b417-b098-4344-b137-362894e4dccc|
|330      |6.228334667224176 |-75.56989476031579|21/06/2024 0

In [227]:
!hdfs dfs -mkdir /silver/new

mkdir: `/silver/new': File exists


# Definir la ruta base en HDFS para la capa Silver

In [228]:
silver_path_hdfs = "hdfs://localhost:9000/silver/new"

In [229]:
# Transformación del DataFrame
df_silver = df_bronze.withColumn("event_date", to_timestamp(col("date"), "dd/MM/yyyy HH:mm:ss")) \
    .withColumn("partition_date", date_format(col("event_date"), "ddMMyyyy")) \
    .withColumn("event_day", dayofmonth(col("event_date"))) \
    .withColumn("event_hour", hour(col("event_date"))) \
    .withColumn("event_minute", minute(col("event_date"))) \
    .withColumn("event_month", month(col("event_date"))) \
    .withColumn("event_second", second(col("event_date"))) \
    .withColumn("event_year", year(col("event_date"))) \
    .withColumnRenamed("order_id", "order_id") \
    .withColumnRenamed("employee_id", "employee_id") \
    .withColumnRenamed("quantity_products", "quantity_products") \
    .withColumnRenamed("latitude", "latitude") \
    .withColumnRenamed("longitude", "longitude") \
    .withColumnRenamed("customer_id", "customer_id")

# Eliminar la columna original de fecha si no es necesaria
df_silver = df_silver.drop("date")

# Mostrar el esquema y las primeras 10 filas
df_silver.printSchema()
df_silver.show(10, truncate=False)

root
 |-- object_id: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- employee_id: integer (nullable = true)
 |-- quantity_products: integer (nullable = true)
 |-- order_id: string (nullable = true)
 |-- event_date: timestamp (nullable = true)
 |-- partition_date: string (nullable = true)
 |-- event_day: integer (nullable = true)
 |-- event_hour: integer (nullable = true)
 |-- event_minute: integer (nullable = true)
 |-- event_month: integer (nullable = true)
 |-- event_second: integer (nullable = true)
 |-- event_year: integer (nullable = true)

+---------+------------------+------------------+-----------+-----------+-----------------+------------------------------------+-------------------+--------------+---------+----------+------------+-----------+------------+----------+
|object_id|latitude          |longitude         |customer_id|employee_id|quantity_products|order_id       

In [231]:
# Guardar el DataFrame df_silver en la ruta definida en HDFS
df_silver.write.mode("append").parquet(silver_path_hdfs + "/df_silver")

In [232]:
# Crear la base de datos silver en Hive
tables_silver = spark.sql("CREATE DATABASE IF NOT EXISTS silver")
tables_silver.show()

++
||
++
++



In [233]:
# Crear la base de datos silver si no existe
spark.sql("CREATE DATABASE IF NOT EXISTS silver")
# Guardar los datos como una tabla en la base de datos silver
df_silver.write.mode('overwrite').saveAsTable('silver.silver_new')
# Verificar que la tabla está guardada en empleados
spark.sql("SELECT * FROM silver.silver_new").show(truncate=False)

+---------+-----------------+------------------+-----------+-----------+-----------------+------------------------------------+-------------------+--------------+---------+----------+------------+-----------+------------+----------+
|object_id|latitude         |longitude         |customer_id|employee_id|quantity_products|order_id                            |event_date         |partition_date|event_day|event_hour|event_minute|event_month|event_second|event_year|
+---------+-----------------+------------------+-----------+-----------+-----------------+------------------------------------+-------------------+--------------+---------+----------+------------+-----------+------------+----------+
|334      |6.222446938596848|-75.56855864645483|8657       |2232       |392              |d8b9b417-b098-4344-b137-362894e4dceu|2024-06-21 01:59:20|21062024      |21       |1         |59          |6          |20          |2024      |
|339      |6.216340691862865|-75.52325545011696|3118       |4942    

In [234]:
df_neighborhoods.printSchema()


root
 |-- OBJECTID: long (nullable = true)
 |-- CODIGO: string (nullable = true)
 |-- NOMBRE: string (nullable = true)
 |-- IDENTIFICACION: string (nullable = true)
 |-- LIMITEMUNICIPIOID: string (nullable = true)
 |-- SUBTIPO_COMUNACORREGIMIENTO: long (nullable = true)
 |-- LINK_DOCUMENTO: integer (nullable = true)
 |-- SHAPEAREA: double (nullable = true)
 |-- SHAPELEN: double (nullable = true)
 |-- geometry: binary (nullable = true)



# Union (join) 

In [235]:
from pyspark.sql.functions import col

# Seleccionar solo las columnas NOMBRE e IDENTIFICACION de df_neighborhoods, y renombrar OBJECTID a object_id
df_neighborhoods_seleccionado = df_neighborhoods.select(
    col("OBJECTID").alias("object_id"),
    col("NOMBRE").alias("neighborhood"),
    col("IDENTIFICACION").alias("commune")
)

# Realizar el left join en la columna object_id
df_resultado = df_silver.join(
    df_neighborhoods_seleccionado,
    df_silver['object_id'] == df_neighborhoods_seleccionado['object_id'],
    "left"
).select(
    df_silver['*'],  # Seleccionar todas las columnas de df_silver
    df_neighborhoods_seleccionado['neighborhood'],  # Seleccionar la columna renombrada neighborhood
    df_neighborhoods_seleccionado['commune']  # Seleccionar la columna renombrada commune
).drop(df_silver['object_id'])  # Eliminar la columna object_id del resultado final

# Mostrar el resultado (opcional)
df_resultado.show()


+------------------+------------------+-----------+-----------+-----------------+--------------------+-------------------+--------------+---------+----------+------------+-----------+------------+----------+--------------------+----------------+
|          latitude|         longitude|customer_id|employee_id|quantity_products|            order_id|         event_date|partition_date|event_day|event_hour|event_minute|event_month|event_second|event_year|        neighborhood|         commune|
+------------------+------------------+-----------+-----------+-----------------+--------------------+-------------------+--------------+---------+----------+------------+-----------+------------+----------+--------------------+----------------+
| 6.272252523303907|-75.64754590534695|       6253|       1679|              225|d8b9b417-b098-434...|2024-06-21 01:55:01|      21062024|       21|         1|          55|          6|           1|      2024|CORREGIMIENTO DE ...|CORREGIMIENTO 60|
| 6.228334667224

In [236]:
# Mostrar el DataFrame
df_resultado.show(truncate=False, n=5)

+------------------+------------------+-----------+-----------+-----------------+------------------------------------+-------------------+--------------+---------+----------+------------+-----------+------------+----------+-------------------------------------+----------------+
|latitude          |longitude         |customer_id|employee_id|quantity_products|order_id                            |event_date         |partition_date|event_day|event_hour|event_minute|event_month|event_second|event_year|neighborhood                         |commune         |
+------------------+------------------+-----------+-----------+-----------------+------------------------------------+-------------------+--------------+---------+----------+------------+-----------+------------+----------+-------------------------------------+----------------+
|6.272252523303907 |-75.64754590534695|6253       |1679       |225              |d8b9b417-b098-4344-b137-362894e4dccc|2024-06-21 01:55:01|21062024      |21       |

In [237]:
df_resultado.printSchema()

root
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- employee_id: integer (nullable = true)
 |-- quantity_products: integer (nullable = true)
 |-- order_id: string (nullable = true)
 |-- event_date: timestamp (nullable = true)
 |-- partition_date: string (nullable = true)
 |-- event_day: integer (nullable = true)
 |-- event_hour: integer (nullable = true)
 |-- event_minute: integer (nullable = true)
 |-- event_month: integer (nullable = true)
 |-- event_second: integer (nullable = true)
 |-- event_year: integer (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- commune: string (nullable = true)

